In [ ]:
print 'Hola!'

from robobrowser import RoboBrowser
from textblob import TextBlob
from pattern.es import tag
urls = 'http://www.elmostrador.cl/','http://www.elmostrador.cl/noticias/pais','http://www.elmostrador.cl/noticias/mundo','http://www.elmostrador.cl/noticias/opinion'
urls = urls[:1]
from pattern.vector import KNN, count
from collections import Counter, defaultdict
import indicoio
indicoio.config.api_key = '37958de57fa5f9d8d4bbec0c0902fd1e'
import networkx as nx

from string import uppercase

def is_person(txt):
    partes = txt.split(' ')
    if len(partes)<2:
        return False
    for parte in partes:
        try:
            if parte[0] not in uppercase:
                return False
        except:
            return False
    return True


browser = RoboBrowser(history=True, parser='html5lib')
G=nx.Graph()
x=0
for url in urls:
    print url
    browser.open(url)
    articulos = browser.select('h3 a') + browser.select('h2 a')
    for articulo in articulos:
        if x>50:
            break
        x+=1
        if '2016' not in articulo.attrs['href']:
            continue
        browser.follow_link(articulo)
        texto = browser.select('.cuerpo-noticia')
        if texto:
            texto = texto[0].text
        else:
            continue        
        titulo = browser.select('.datos-noticias h2')
        if titulo:
            titulo = titulo[0].text
        else:
            continue
        try:
            people = indicoio.people(texto)
        except:
            continue
        palabras = []
        for person in people:
            nombre = person['text']
            if is_person(nombre):
                #print nombre, person['confidence']
                palabras.append(nombre)
        for palabra1 in palabras:
            for palabra2 in palabras:
                if palabra1 != palabra2:
                    G.add_edge(palabra1, palabra2)



        


In [ ]:
import json
from networkx.readwrite import json_graph
 
for n in G:
    G.node[n]['name'] = n
 
d = json_graph.node_link_data(G)
json.dump(d, open('force/force.json','w'))

In [ ]:
import flask
app = flask.Flask(__name__, static_folder="force")

@app.route('/<path:path>')
def static_proxy(path):
  return app.send_static_file(path)
print('\nGo to http://localhost:8000/force.html to see the example\n')
app.run(port=8000)